### Import libraries

In [54]:
import pandas as pd
from sqlalchemy import create_engine
from json import loads
import plotly.express as px

### Loading and filtering data

In [55]:
is_load_csv = True
if not is_load_csv:
    disk_engine = create_engine("{server}+{dialect}://{username}:{password}@{host}:{port}/{database}".format(
        server='mysql',
        dialect='pymysql',
        host='localhost',
        username='root',
        password='password',
        port='3306',
        database='web_scraping_project'
    ), pool_recycle=14400)

    df = pd.read_sql_table('jobs_offers', disk_engine)
else:
    df = pd.read_csv('jobs_offers.csv', encoding='utf-8')
    # convert str (json in database) to python list with loads function
    df.technologies = df.technologies.apply(lambda str_list: loads(str_list))

# set job_offer_id as default index
df.set_index('job_offer_id', inplace=True)
# delete unnecessary columns
df.drop(['description', 'company_url', 'date_time', 'criteria'], axis=1, inplace=True)
# technologies filter
technologies_filter = df.technologies.apply(lambda l: len(l) > 0)
# apply technologies filter to df
df = df[technologies_filter]
df.shape

(354, 6)

### Creation of a DataFrame of European countries

In [56]:
# 18 countries selected for EU
union_european_countries = [
    'FRANCE', 'GERMANY', 'BELGIUM', 'DENMARK', 'ESTONIA',
    'FINLAND', 'GREECE', 'ICELAND', 'IRELAND', 'ITALY',
    'LUXEMBOURG', 'NETHERLANDS', 'NORWAY', 'POLAND',
    'PORTUGAL', 'SPAIN', 'SWEDEN', 'SWITZERLAND',
]
# create DataFrame by filtering only countries listed in union_european_countries list
df_eu = df.query('country in @union_european_countries')
df_eu.shape

(193, 6)

In [57]:
all_tech = []
for tech in df.technologies:
    all_tech.extend(tech)
freq_technologies = pd.Series(all_tech).value_counts(normalize=True)
# print top 10
freq_technologies.head(10)

React         0.102585
Java          0.084237
C#            0.072560
Angular       0.068390
.NET          0.065054
Python        0.060884
Git           0.058382
TypeScript    0.040867
PHP           0.030025
Node.js       0.028357
dtype: float64

In [58]:
fig_top_25_tech = px.bar(
    freq_technologies.head(25),
    title=f"Top 25 technologies used in EU web projects, number of job offers: {df_eu.shape[0]}",
    template='plotly_dark',
    labels=dict(value='Frequency', index='Technology')
)
fig_top_25_tech.show()